In [ ]:
import requests
from bs4 import BeautifulSoup as soup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

def lap():
    base_url = "https://www.flipkart.com/search?q={}&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={}"
    x=input("Enter the item you want to buy: ")
    with open('item_prices.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Item Name', 'Price'])
    
        for page in range(1, 30): 
            url = base_url.format(x,page)
            response = requests.get(url)
            soupdata = soup(response.content)

            items = soupdata.find_all('div', {'class': '_2kHMtA'})

            for item in items:
                name = item.find('div', {'class': '_4rR01T'}).get_text().strip()
                pr1 = item.find('div', {'class': '_30jeq3 _1_WHN1'}).get_text().strip()
                price = pr1.replace('₹','').replace(',','').strip()
                writer.writerow([name, price])
            
    df = pd.read_csv('item_prices.csv')
    df['Price'] = pd.to_numeric(df['Price'])
    plt.figure(figsize=(15, 8))

    # create a list of serial numbers for each item
    serial_numbers = range(1, len(df)+1)

    # plot each subplot with corresponding serial numbers and prices
    for i in range(5):
        start_idx = i * 50
        end_idx = (i+1) * 50 if i != 4 else len(df)
        plt.subplot(6, 5, i+1)
        plt.scatter(serial_numbers[start_idx:end_idx], df['Price'][start_idx:end_idx], s=50, alpha=0.5)
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=12)
        plt.ylabel('Price (INR)', fontsize=12)
        plt.title(f'Subplot {i+1}', fontsize=14)
        plt.grid()

    plt.tight_layout()
    plt.show()

        
    # print serial number and corresponding laptop name from csv file
    with open('item_prices.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # skip header row
        for i, row in enumerate(reader, start=1):
            print(f'{i}. {row[0]}')

def gold():
    # Make a request to the webpage
    url = 'https://www.goodreturns.in/gold-rates/#Todays+Gold+Rate+in+Major+Countries'
    response = requests.get(url)

    # Parse the HTML page using BeautifulSoup library
    soupdata = soup(response.content)

    # Find the table containing the gold rates
    table = soupdata.find_all('div', {'class': 'gold_silver_table'})[3]

    # Find the rows in the table
    rows = table.find_all('tr')

    # Define currency exchange rates as dictionary
    # Fetch live prices from Open Exchange Rates API
    response = requests.get(f"https://openexchangerates.org/api/latest.json?app_id=69ef804868e94c86bf2120ddd2e9a2c9")
    
    if response.status_code == 200:
        data = response.json()
        # get the rates data from the response
        rates = data['rates']
        # add USD rate with value 1.0 to the rates dictionary
        rates['USD'] = 1.0
        # create a dictionary with the desired format
        currency_rates = {'INR': rates['INR'], 'USD': rates['USD'], 'BHD': rates['BHD'], 'KWD': rates['KWD'], 'MYR': rates['MYR'], 'OMR': rates['OMR'], 'QAR': rates['QAR'], 'SGD': rates['SGD'], 'AED': rates['AED'], 'SAR': rates['SAR']}
        print(currency_rates)
    else:
        print("Failed to retrieve data from Open Exchange Rates API")
    currency_rates = {'INR': 82.2, 'USD': 1.0,'BHD': 0.38,'KWD': 0.31,'MYR': 4.44,'OMR': 0.38,'QAR': 3.64,'SGD': 1.34,'AED': 3.67,'SAR': 3.75}

    # Create a new CSV file and write the header row
    with open('gold_rates.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Country', '22k Gold Rate', 'USD rates'])

        # Loop through each row and extract the country name and 22k gold rate
        for row in rows[1:]:
            columns = row.find_all('td')
            country = columns[0].text.strip()
        
            # Remove commas and convert gold rate to float
            gold_rate_str = columns[1].text.strip().replace(',', '')
            gold_rate = float(gold_rate_str[3:])

            # Convert gold rate to USD
            currency_code = gold_rate_str[:3]
            usd_rate = gold_rate / currency_rates[currency_code]

            # Write the country name and 22k gold rate (in USD) to the CSV file
            writer.writerow([country,gold_rate_str,usd_rate])
            
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv('gold_rates.csv')
    gold_rates = np.array(df['USD rates'])

    # Create a scatter plot of countries and their corresponding gold rates in USD
    plt.scatter(df['Country'], df['USD rates'])

    # Set the x-axis label to 'Country'
    plt.xlabel('Country')

    # Set the y-axis label to 'Gold Rate (USD)'
    plt.ylabel('Gold Rate (USD)')

    # Set the title of the plot to 'Gold Rates by Country (USD)'
    plt.title('Gold Rates by Country (USD)')

    # Rotate the x-axis labels by 90 degrees to make them easier to read
    plt.xticks(rotation=90)


    plt.plot(gold_rates,linewidth='3',label='prices')
    plt.legend()

    # Show the plot
    plt.show()
    
    
x=int(input("Enter 1 for any item\n\t  2 for gold:\n"))
if x==1:
    lap()
else:
    gold()